# Segmenting neighborhoods in Toronto



Import needed dependencies.


In [1]:
#Web scrapping stage
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

import numpy as np # library to handle data in a vectorized manner

import lxml
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>



## 1. Get Toronto's neighborhoods


The Wikipedia article containing Toronto's neighborhoods is: https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto

From there I need to get the names.

To download the contents of the webpage `url` in text format and store in a variable called `data` and create a `BeautifulSoup` object using the BeautifulSoup constructor:

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en"><head>
<meta charset="utf-8"/>
<title>List of neighbourhoods in Toronto - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"6ea3ce13-a2cb-45a5-8e71-00409e00e620","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighbourhoods_in_Toronto","wgTitle":"List of neighbourhoods in Toronto","wgCurRevisionId":1013104083,"wgRevisionId":1013104083,"wgArticleId":1150939,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Articles with short description","Short description is different from Wi

**Explore the data**

Table with district numbers.

In [3]:
table = soup.find_all("table",{"class":"wikitable sortable"})
table = table[1]
table

<table class="wikitable sortable">

<tbody><tr bgcolor="lightblue">
<th width="5%">CDN number
</th>
<th width="20%">City-designated neighbourhood
</th>
<th width="10%">Former city/borough
</th>
<th width="50%">Neighbourhoods covered
</th>
<th width="15%">Map
</th></tr>
<tr>
<td>129
</td>
<td>Agincourt North
</td>
<td>Scarborough
</td>
<td><a href="/wiki/Agincourt,_Toronto" title="Agincourt, Toronto">Agincourt</a> and Brimwood
</td>
<td>
</td></tr>
<tr>
<td>128
</td>
<td>Agincourt South-Malvern West
</td>
<td>Scarborough
</td>
<td><a href="/wiki/Agincourt,_Toronto" title="Agincourt, Toronto">Agincourt</a> and <a href="/wiki/Malvern,_Toronto" title="Malvern, Toronto">Malvern</a>
</td>
<td><a class="image" href="/wiki/File:Malvern_to_locator.gif"><img alt="Malvern to locator.gif" data-file-height="149" data-file-width="291" decoding="async" height="51" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Malvern_to_locator.gif/100px-Malvern_to_locator.gif" srcset="//upload.wikimedia.o

Get table rows' attributes to a DataFrame:

In [4]:
df = pd.read_html(str(table), flavor='html5lib')
df = df[0]
df.head()

,CDN number,City-designated neighbourhood,Former city/borough,Neighbourhoods covered,Map,Unnamed: 5
0,129,Agincourt North,Scarborough,Agincourt and Brimwood,NaN,NaN
1,128,Agincourt South-Malvern West,Scarborough,Agincourt and Malvern,NaN,NaN
2,20,Alderwood,Etobicoke,Alderwood,NaN,NaN
3,95,Annex,Old City of Toronto,The Annex and Seaton Village,NaN,NaN
4,42,Banbury-Don Mills,North York,Don Mills,NaN,NaN


#### Use geopy library to get the latitude and longitude values of New York City.


To get the latitude and logitude of Toronto, I defined a `user_agent` named `to_explorer`.



In [5]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
toronto_latitude = location.latitude
toronto_longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_latitude, toronto_longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [6]:
def get_coordinates(neighbourhood, borough):
    address = str(neighbourhood)+", "+borough+',Toronto, Ontario, Canada'
    location = geolocator.geocode(address)
    if location != None:
        latitude = location.latitude
        longitude = location.longitude
        return [latitude, longitude]
    else:
        neighbourhoods = neighbourhood.split('-')
        for neighbourhood in neighbourhoods:
            address = str(neighbourhood)+","+borough+", Toronto, Ontario, Canada"
            location = geolocator.geocode(address)
            if location != None:
                latitude = location.latitude
                longitude = location.longitude
                return [latitude, longitude]
        address = borough+", Toronto, Ontario, Canada"
        location = geolocator.geocode(address)
        if location != None:
                latitude = location.latitude
                longitude = location.longitude
                return [latitude, longitude]
        return [None, None]

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
for row in df.values:
    #print(row)
    borough = str(row[2])
    neighbourhood = str(row[1])
    #rint(borough+" "+neighbourhood)
    
    #Get latitude and longitude
    [latitude, longitude] = get_coordinates(neighbourhood, borough)
    neighborhoods = neighborhoods.append({'Borough': borough,
                                            'Neighborhood': neighbourhood,
                                            'Latitude': latitude,
                                            'Longitude': longitude}, ignore_index=True)
    

In [9]:
neighborhoods.head(140)

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Agincourt North,43.808038,-79.266439
1,Scarborough,Agincourt South-Malvern West,43.781969,-79.257689
2,Etobicoke,Alderwood,43.601717,-79.545232
3,Old City of Toronto,Annex,43.670338,-79.407117
4,North York,Banbury-Don Mills,43.752339,-79.365716
5,North York,Bathurst Manor,43.763893,-79.456367
6,Old City of Toronto,Bay Street Corridor,43.672135,-79.390499
7,North York,Bayview Village,43.769197,-79.376662
8,North York,Bayview Woods-Steeles,43.798127,-79.382973
9,North York,Bedford Park-Nortown,43.737388,-79.410925


As verification, make sure that the dataset has all 6 districts and 140 neighborhoods.


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 6 boroughs and 140 neighborhoods.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<a id='item2'></a>


## 2. Explore Toronto's Neighborhoods

In [12]:
CLIENT_ID = 'QDJM0CJEOATSCZCV15MHQEVTOWIFENTNDQ2533RBDQWM4L4K' # your Foursquare ID
CLIENT_SECRET = 'DAHZBZYEATI35XPLG51AT4SHFGY0D5RPFO53X34MGWLLYHSJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QDJM0CJEOATSCZCV15MHQEVTOWIFENTNDQ2533RBDQWM4L4K
CLIENT_SECRET:DAHZBZYEATI35XPLG51AT4SHFGY0D5RPFO53X34MGWLLYHSJ


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_venues_.

In [14]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Agincourt North
Agincourt South-Malvern West
Alderwood
Annex
Banbury-Don Mills
Bathurst Manor
Bay Street Corridor
Bayview Village
Bayview Woods-Steeles
Bedford Park-Nortown
Beechborough-Greenbrook
Bendale
Birchcliffe-Cliffside
Black Creek
Blake-Jones
Briar Hill-Belgravia
Bridle Path-Sunnybrook-York Mills
Broadview North
Brookhaven-Amesbury
Cabbagetown-South St. James Town
Caledonia-Fairbank
Casa Loma
Centennial Scarborough
Church-Yonge Corridor
Clairlea-Birchmount
Clanton Park
Cliffcrest
Corso Italia-Davenport
Crescent Town
Danforth - East York
Danforth Village - Toronto
Don Valley Village
Dorset Park
Dovercourt-Wallace Emerson-Junction
Downsview-Roding-CFB
Dufferin Grove
East End-Danforth
Edenbridge-Humber Valley
Eglinton East
Elms-Old Rexdale
Englemount-Lawrence
Eringate-Centennial-West Deane
Etobicoke West Mall
Flemingdon Park
Forest Hill North
Forest Hill South
Glenfield-Jane Heights
Greenwood-Coxwell
Guildwood
Henry Farm
High Park North
High Park-Swansea
Highland Creek
Hillcrest V

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(3147, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt North,43.808038,-79.266439,Menchie's,43.808338,-79.268288,Frozen Yogurt Shop
1,Agincourt North,43.808038,-79.266439,Saravanaa Bhavan South Indian Restaurant,43.810117,-79.269275,Indian Restaurant
2,Agincourt North,43.808038,-79.266439,Shoppers Drug Mart,43.808894,-79.269854,Pharmacy
3,Agincourt North,43.808038,-79.266439,Booster Juice,43.809915,-79.269382,Juice Bar
4,Agincourt North,43.808038,-79.266439,Dollarama,43.808894,-79.269854,Discount Store


**how many venues were returned for each neighborhood?**

In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt North,27,27,27,27,27,27
Agincourt South-Malvern West,14,14,14,14,14,14
Alderwood,10,10,10,10,10,10
Annex,36,36,36,36,36,36
Banbury-Don Mills,3,3,3,3,3,3
Bathurst Manor,4,4,4,4,4,4
Bay Street Corridor,100,100,100,100,100,100
Bayview Village,11,11,11,11,11,11
Bayview Woods-Steeles,2,2,2,2,2,2


#### Each neighborhood

In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorsports Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Martial Arts School,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motorsports Shop,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt North,0.000000,0.000000,0.000000,0.000000,0.0,0

print each neighborhood along with the top 5 most common venues

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt North----
                 venue  freq
0                 Bank  0.07
1               Bakery  0.07
2  Fried Chicken Joint  0.04
3           Beer Store  0.04
4          Men's Store  0.04


----Agincourt South-Malvern West----
                  venue  freq
0            Restaurant  0.29
1        Breakfast Spot  0.07
2  Gym / Fitness Center  0.07
3           Coffee Shop  0.07
4     Electronics Store  0.07


----Alderwood----
            venue  freq
0     Pizza Place   0.2
1        Pharmacy   0.1
2    Skating Rink   0.1
3  Sandwich Place   0.1
4             Gym   0.1


----Annex----
              venue  freq
0       Pizza Place  0.08
1     Grocery Store  0.06
2            Bistro  0.06
3   Thai Restaurant  0.06
4  Ramen Restaurant  0.03


----Banbury-Don Mills----
          venue  freq
0   Gas Station  0.33
1          Park  0.33
2  Intersection  0.33
3   Yoga Studio  0.00
4  Optical Shop  0.00


----Bathurst Manor----
               venue  freq
0         Playground  0.25
1  Conve

                  venue  freq
0  Fast Food Restaurant  0.21
1           Pizza Place  0.14
2              Pharmacy  0.14
3                  Park  0.07
4         Grocery Store  0.07


----Maple Leaf----
              venue  freq
0            Bakery  0.33
1  Basketball Court  0.33
2             Trail  0.33
3       Yoga Studio  0.00
4      Noodle House  0.00


----Markland Wood----
            venue  freq
0  Baseball Field  0.25
1     Golf Course  0.25
2       Piano Bar  0.25
3            Park  0.25
4      Non-Profit  0.00


----Milliken----
                 venue  freq
0   Chinese Restaurant  0.13
1  Japanese Restaurant  0.08
2               Bakery  0.08
3      Bubble Tea Shop  0.05
4    Food & Drink Shop  0.05


----Mimico----
          venue  freq
0   Yoga Studio   0.2
1         Diner   0.2
2        Bakery   0.2
3           Bar   0.2
4  Skating Rink   0.2


----Morningside----
                  venue  freq
0                  Park  0.18
1           Coffee Shop  0.18
2           Supermark

                  venue  freq
0     Indian Restaurant  0.31
1  Caribbean Restaurant  0.15
2                  Bank  0.08
3        Ice Cream Shop  0.08
4              Pharmacy  0.08


----Thorncliffe Park----
                venue  freq
0         Coffee Shop  0.08
1   Afghan Restaurant  0.08
2  Turkish Restaurant  0.08
3      Sandwich Place  0.08
4   Indian Restaurant  0.08


----Trinity-Bellwoods----
                   venue  freq
0           Cocktail Bar  0.07
1                 Bakery  0.07
2              Pet Store  0.05
3                   Café  0.05
4  Vietnamese Restaurant  0.05


----University----
                 venue  freq
0                 Café  0.17
1   Italian Restaurant  0.07
2  Japanese Restaurant  0.07
3            Bookstore  0.07
4                 Park  0.07


----Victoria Village----
                       venue  freq
0                        Spa   0.2
1            Thai Restaurant   0.2
2  Middle Eastern Restaurant   0.2
3                   Bus Line   0.2
4   Mediterran

function to sort the venues in descending order.

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Bank,Bakery,Movie Theater,Frozen Yogurt Shop,Beer Store,Chinese Restaurant,Fried Chicken Joint,Juice Bar,Spa,Restaurant
1,Agincourt South-Malvern West,Restaurant,Breakfast Spot,Sporting Goods Shop,Japanese Restaurant,Gym / Fitness Center,Salad Place,Intersection,Coffee Shop,Bookstore,Department Store
2,Alderwood,Pizza Place,Skating Rink,Sandwich Place,Pub,Gym,Pool,Coffee Shop,Dance Studio,Pharmacy,Fast Food Restaurant
3,Annex,Pizza Place,Thai Restaurant,Bistro,Grocery Store,Fast Food Restaurant,Korean Restaurant,Fried Chicken Joint,Bookstore,Park,Burrito Place
4,Banbury-Don Mills,Intersection,Gas Station,Park,Egyptian Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Women's Store


## 3. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 3, 3, 1, 1, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,Agincourt North,43.808038,-79.266439,1,Bank,Bakery,Movie Theater,Frozen Yogurt Shop,Beer Store,Chinese Restaurant,Fried Chicken Joint,Juice Bar,Spa,Restaurant
1,Scarborough,Agincourt South-Malvern West,43.781969,-79.257689,1,Restaurant,Breakfast Spot,Sporting Goods Shop,Japanese Restaurant,Gym / Fitness Center,Salad Place,Intersection,Coffee Shop,Bookstore,Department Store
2,Etobicoke,Alderwood,43.601717,-79.545232,1,Pizza Place,Skating Rink,Sandwich Place,Pub,Gym,Pool,Coffee Shop,Dance Studio,Pharmacy,Fast Food Restaurant
3,Old City of Toronto,Annex,43.670338,-79.407117,1,Pizza Place,Thai Restaurant,Bistro,Grocery Store,Fast Food Restaurant,Korean Restaurant,Fried Chicken Joint,Bookstore,Park,Burrito Place
4,North York,Banbury-Don Mills,43.752339,-79.365716,3,Intersection,Gas Station,Park,Egyptian Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Women's Store


Finally, let's visualize the resulting clusters


In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Blake-Jones,Gym,Park,Women's Store,Dumpling Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
22,Centennial Scarborough,Park,Women's Store,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field,Eastern European Restaurant
41,Eringate-Centennial-West Deane,Park,Women's Store,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field,Eastern European Restaurant
64,Kingsview Village-The Westway,Park,Women's Store,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field,Eastern European Restaurant
97,Pelmo Park-Humberlea,Coffee Shop,Park,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Women's Store
125,Westminster-Branson,Park,Coffee Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Women's Store


#### Cluster 2


In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Bank,Bakery,Movie Theater,Frozen Yogurt Shop,Beer Store,Chinese Restaurant,Fried Chicken Joint,Juice Bar,Spa,Restaurant
1,Agincourt South-Malvern West,Restaurant,Breakfast Spot,Sporting Goods Shop,Japanese Restaurant,Gym / Fitness Center,Salad Place,Intersection,Coffee Shop,Bookstore,Department Store
2,Alderwood,Pizza Place,Skating Rink,Sandwich Place,Pub,Gym,Pool,Coffee Shop,Dance Studio,Pharmacy,Fast Food Restaurant
3,Annex,Pizza Place,Thai Restaurant,Bistro,Grocery Store,Fast Food Restaurant,Korean Restaurant,Fried Chicken Joint,Bookstore,Park,Burrito Place
6,Bay Street Corridor,Italian Restaurant,Café,Coffee Shop,Boutique,French Restaurant,Clothing Store,Spa,Hotel,Indian Restaurant,Gym
7,Bayview Village,Bank,Pizza Place,Outdoor Supply Store,Fish Market,Sporting Goods Shop,Fast Food Restaurant,Gas Station,Sandwich Place,Breakfast Spot,Persian Restaurant
8,Bayview Woods-Steeles,Coffee Shop,Dog Run,Egyptian Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Women's Store
10,Beechborough-Greenbrook,Furniture / Home Store,Convenience Store,Dessert Shop,Video Store,Discount Store,Bar,Auto Garage,Italian Restaurant,Fast Food Restaurant,Ethiopian Restaurant
12,Birchcliffe-Cliffside,Pizza Place,Coffee Shop,Snack Place,Sandwich Place,Grocery Store,Pub,Breakfast Spot,Pharmacy,Dessert Shop,Cupcake Shop
13,Black Creek,Playground,Fast Food Restaurant,Coffee Shop,Intersection,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Field,Eastern European Restaurant


#### Cluster 3


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Bedford Park-Nortown,Rental Car Location,Women's Store,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field,Eastern European Restaurant


#### Cluster 4


In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Banbury-Don Mills,Intersection,Gas Station,Park,Egyptian Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Women's Store
5,Bathurst Manor,Playground,Park,Convenience Store,Baseball Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
28,Crescent Town,Golf Course,Park,Convenience Store,Metro Station,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
40,Englemount-Lawrence,Playground,Coffee Shop,Gym,Park,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
44,Forest Hill North,Playground,Park,Bank,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Field,Eastern European Restaurant
45,Forest Hill South,Playground,Park,Bank,Flea Market,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Field,Eastern European Restaurant
49,Henry Farm,Park,Intersection,Tennis Court,Lawyer,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Women's Store
50,High Park North,Park,Tennis Court,Baseball Field,Gym / Fitness Center,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Women's Store
52,Highland Creek,IT Services,Construction & Landscaping,Park,Women's Store,Fast Food Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
55,Humber Summit,Flower Shop,Furniture / Home Store,Park,Bakery,Construction & Landscaping,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Eastern European Restaurant


#### Cluster 5


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Bendale,Dog Run,Grocery Store,Department Store,Chinese Restaurant,Discount Store,Fast Food Restaurant,Women's Store,Event Space,Falafel Restaurant,Farmers Market
37,Edenbridge-Humber Valley,Fast Food Restaurant,Indian Restaurant,Bus Stop,Park,Women's Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field
62,Kennedy Park,Fast Food Restaurant,Chinese Restaurant,Asian Restaurant,Grocery Store,Fish Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant,Field,Flea Market
74,Malvern,Fast Food Restaurant,Pizza Place,Pharmacy,Sandwich Place,Bubble Tea Shop,Skating Rink,Park,Gym / Fitness Center,Supermarket,Grocery Store
102,Rexdale-Kipling,Pizza Place,Department Store,Fast Food Restaurant,Sandwich Place,Fish Market,Fish & Chips Shop,Flea Market,Financial or Legal Service,Filipino Restaurant,Eastern European Restaurant
106,Rouge,Fast Food Restaurant,Park,Caribbean Restaurant,Women's Store,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Field
108,Rustic,Fast Food Restaurant,Caribbean Restaurant,Gas Station,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Women's Store,Food & Drink Shop
